In [1]:
%run utils/devtools.py

In [2]:
%reloadmypkg utils

import pandas as pd
from utils.url2platform import *
from utils.standardise_url import *
from utils.fetch_info import *
from utils.dump_df_to_json import *

✅ Reloaded package 'utils' and its submodules.


In [3]:
sheet_name = "../raw-data/neetcode/250-sheet.json"
export_file = "../cleaned-data/neetcode/250.json"

In [4]:
with open(sheet_name) as file:
    raw = json.load(file)

df = pd.DataFrame(raw, columns=['url'])

In [5]:
df.head(2)

,url
0,https://leetcode.com/problems/concatenation-of...
1,https://leetcode.com/problems/contains-duplicate/


In [6]:
df.shape

(250, 1)

In [7]:
# Find the duplicates

dupes_df = df[df.duplicated(subset="url", keep=False)]
dupes_df.shape

(0, 1)

In [8]:
import nest_asyncio
nest_asyncio.apply()

standardised_df = await standardise_urls_async(df, "neetcode-250-urls-expanded.json")

Filter list is empty. Trying to expand all 250 URLs.

Processing batch 1/2
Loaded 0 results from checkpoint
Processing 200 pending URLs in this batch


Expanding URLs: 100%|██████████| 200/200 [00:02<00:00, 96.91it/s] 


Progress saved to neetcode-250-urls-expanded.json
----------------------------------------------------------------------
Progress saved to neetcode-250-urls-expanded.json
----------------------------------------------------------------------

Processing batch 2/2
Loaded 200 results from checkpoint
Processing 50 pending URLs in this batch


Expanding URLs: 100%|██████████| 50/50 [00:00<00:00, 64.03it/s]

Progress saved to neetcode-250-urls-expanded.json
----------------------------------------------------------------------
Progress saved to neetcode-250-urls-expanded.json
----------------------------------------------------------------------

250/250 URLs expanded in 6.82 seconds.


In [9]:
standardised_df.shape

(250, 3)

In [10]:
# Adding type column
standardised_df['type'] = standardised_df['expanded_stripped_url'].apply(
    lambda url: "problem" if is_problem_url(url) else "tutorial"
)

In [14]:
# Adding id_base, id, title, platform columns
import nest_asyncio
nest_asyncio.apply()

final_df = await fetch_info(standardised_df, "neetcode-250-info-added.json", resume=True)

Found 250 URLs.

Batch 1/2: items 0-199
Loaded 249 from checkpoint
Processing 1 URLs...


Fetching items:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching items: 100%|██████████| 1/1 [00:00<00:00,  1.26it/s]

Saved progress (250 items) to neetcode-250-info-added.json
------------------------------
Saved progress (250 items) to neetcode-250-info-added.json
------------------------------

Batch 2/2: items 200-249
Loaded 250 from checkpoint
Nothing new in this batch
Saved progress (250 items) to neetcode-250-info-added.json
------------------------------

Fetched 250/250 items in 0.9s.


In [15]:
final_df.shape

(250, 8)

In [16]:
dump_df_to_json(final_df, export_file)